### Original code from the artical

In [1]:
from random import randint
from numpy import array
from numpy import argmax

 
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
    return [randint(0, 99) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]
 
# generate random sequence
sequence = generate_sequence()
print(sequence)
# one hot encode
encoded = one_hot_encode(sequence)
print(encoded)
# one hot decode
decoded = one_hot_decode(encoded)
print(decoded)

[13, 2, 22, 45, 62, 26, 44, 98, 60, 83, 62, 59, 97, 26, 50, 25, 23, 29, 97, 43, 71, 33, 72, 95, 16]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[13, 2, 22, 45, 62, 26, 44, 98, 60, 83, 62, 59, 97, 26, 50, 25, 23, 29, 97, 43, 71, 33, 72, 95, 16]


In [2]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# generate data for the lstm
def generate_data():
    # generate sequence
    sequence = generate_sequence()
    # one hot encode
    encoded = one_hot_encode(sequence)
    # convert to 3d for input
    X = encoded.reshape(encoded.shape[0], 1, encoded.shape[1])
    return X, encoded
 
# define model
model = Sequential()
model.add(LSTM(15, input_shape=(1, 100)))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
for i in range(500):
    X, y = generate_data()
    model.fit(X, y, epochs=1, batch_size=1, verbose=2)
# evaluate model on new data
X, y = generate_data()
yhat = model.predict(X)
print('Expected:  %s' % one_hot_decode(y))
print('Predicted: %s' % one_hot_decode(yhat))

25/25 - 1s - loss: 4.6099 - accuracy: 0.0000e+00 - 740ms/epoch - 30ms/step
25/25 - 0s - loss: 4.6102 - accuracy: 0.0000e+00 - 19ms/epoch - 761us/step
25/25 - 0s - loss: 4.5981 - accuracy: 0.0000e+00 - 20ms/epoch - 801us/step
25/25 - 0s - loss: 4.5880 - accuracy: 0.0000e+00 - 19ms/epoch - 761us/step
25/25 - 0s - loss: 4.5871 - accuracy: 0.0800 - 18ms/epoch - 721us/step
25/25 - 0s - loss: 4.5936 - accuracy: 0.0000e+00 - 19ms/epoch - 751us/step
25/25 - 0s - loss: 4.5845 - accuracy: 0.0800 - 19ms/epoch - 761us/step
25/25 - 0s - loss: 4.6138 - accuracy: 0.0400 - 18ms/epoch - 721us/step
25/25 - 0s - loss: 4.5993 - accuracy: 0.0000e+00 - 18ms/epoch - 721us/step
25/25 - 0s - loss: 4.6076 - accuracy: 0.0800 - 18ms/epoch - 721us/step
25/25 - 0s - loss: 4.5994 - accuracy: 0.0400 - 19ms/epoch - 761us/step
25/25 - 0s - loss: 4.5755 - accuracy: 0.0400 - 19ms/epoch - 761us/step
25/25 - 0s - loss: 4.5541 - accuracy: 0.1200 - 19ms/epoch - 761us/step
25/25 - 0s - loss: 4.5664 - accuracy: 0.0800 - 18ms/e

### PyTorch

In [3]:
from torch import nn
import torch

from sklearn.metrics import accuracy_score

In [4]:
class CustomLSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(CustomLSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        
        self.fc = nn.Linear(hidden_size, num_classes) 
    
    def forward(self,x):
        output, (hn, cn) = self.lstm(x)
        hn = hn.view(-1, self.hidden_size) 
        out = self.fc(hn) 
        return out

In [5]:
lstm_net = CustomLSTM(num_classes=100, input_size=100, hidden_size=15, num_layers=1)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_net.parameters(), lr = 1e-3)
n_epochs = 700

In [6]:
for epoch in range(n_epochs):
    X, y = generate_data()
    X = torch.FloatTensor(X)
    y = torch.FloatTensor(y)
    
    optimizer.zero_grad() # Устанавливает градиенты всех оптимизированных равными нулю.
    output = lstm_net.forward(X)
            
    _, predicted = torch.max(output.detach(), 1)
    loss = criterion(output, y)
    
    loss.backward() # Вычисляет градиент текущего тензора
    optimizer.step() #Обновляем параметры оптимайзера
    
    if epoch % 20 == 0: # Каждую 20 эпоху вычиcляем accuracy
        accuracy_train = accuracy_score(predicted, one_hot_decode(y))
    
        print(f'epoch: {epoch}, accuracy: {accuracy_train}')

epoch: 0, accuracy: 0.0
epoch: 20, accuracy: 0.0
epoch: 40, accuracy: 0.08
epoch: 60, accuracy: 0.0
epoch: 80, accuracy: 0.0
epoch: 100, accuracy: 0.0
epoch: 120, accuracy: 0.08
epoch: 140, accuracy: 0.0
epoch: 160, accuracy: 0.12
epoch: 180, accuracy: 0.16
epoch: 200, accuracy: 0.32
epoch: 220, accuracy: 0.2
epoch: 240, accuracy: 0.4
epoch: 260, accuracy: 0.36
epoch: 280, accuracy: 0.48
epoch: 300, accuracy: 0.32
epoch: 320, accuracy: 0.32
epoch: 340, accuracy: 0.72
epoch: 360, accuracy: 0.64
epoch: 380, accuracy: 0.8
epoch: 400, accuracy: 0.8
epoch: 420, accuracy: 0.72
epoch: 440, accuracy: 0.92
epoch: 460, accuracy: 0.84
epoch: 480, accuracy: 0.84
epoch: 500, accuracy: 0.84
epoch: 520, accuracy: 0.96
epoch: 540, accuracy: 0.96
epoch: 560, accuracy: 1.0
epoch: 580, accuracy: 1.0
epoch: 600, accuracy: 1.0
epoch: 620, accuracy: 1.0
epoch: 640, accuracy: 0.88
epoch: 660, accuracy: 1.0
epoch: 680, accuracy: 0.96
